In [1]:
import sys
sys.path.append('../ml_utils')

import data_utils

import vaex
from vaex.ui.colormaps import cm_plusmin

import timeit
import pandas as pd
import numpy as np
import pylab as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
base_dir = './../shared/data/'

## Convert the pandas dataframe to a vaex dataframe

1. Before we remove all the NaN values
2. We convert the pandas dataframe
3. We persist the vaex dataframe in hdf5 format for more performance

In [3]:
all_columns = ['time_stamp','label_nr','date_day','date_month','date_month_nr','date_year','date_hour','date_minute',
           'date_second','transaction_id','tracking_ids','src_ip','src_software_name','src_operating_system_name',
           'src_software_type','src_software_sub_type','src_hardware_type','src_hardware_sub_type','dest_ip',
           'uri_path','http_method','request_operation','request_protocol','http_referrer','response_status',
           'response_status_code','response_detail_reason','response_time_ms','oidc_response_type','oidc_acr_values',
           'oidc_login_hint','oidc_client_id','client_type','oidc_scopes','oidc_ui_locales','ido_id','ido_type',
           'ido_email','client_name','am_realm','am_component','loc_country','loc_region','loc_city','loc_country_code',
           'loc_latitude','loc_longitude','loc_zip_code','loc_time_zone','ldap_user','user_agent','http_query_parameters']

In [4]:
unused_columns = ['transaction_id','tracking_ids','http_referrer','user_agent','http_query_parameters']

In [5]:
columns = all_columns
for col in unused_columns:
    columns.remove(col)

In [6]:
log_data = base_dir + 'swissid_oidc_logs_april_to_sept_2019.zip'

start = timeit.default_timer()
pandas_df = pd.read_csv(log_data, compression='zip', names=columns, header=0, sep='\t', quotechar='"', index_col=None)
stop = timeit.default_timer()

time_consumed = stop - start

print("Original data frame size: {}\n".format(pandas_df.shape))
print("Time: {} seconds\n".format(time_consumed))
print("Time: {} minutes\n".format(time_consumed/60))

Original data frame size: (23154543, 47)

Time: 186.650869475 seconds

Time: 3.1108478245833333 minutes



In [7]:
#display(pandas_df)

In [8]:
# show all columns having NaN
pandas_df.columns[pandas_df.isna().any()].tolist()

['loc_country_code']

In [9]:
# Namibia has country code NA => NaN!!!
nan_df = pandas_df[pandas_df.isna().any(axis=1)]
#nan_df

In [10]:
pandas_df['loc_country_code'].fillna('NA', inplace=True)

In [11]:
# show all columns having NaN
pandas_df.columns[pandas_df.isna().any()].tolist()

[]

In [12]:
# check visually if all is fine now
#pandas_df[pandas_df['loc_country'] == 'Namibia']

In [29]:
# Importing the data from pandas to vaex
start = timeit.default_timer()
vaex_df = vaex.from_pandas(pandas_df)
stop = timeit.default_timer()

print("Time: {} minutes\n".format((stop - start)/60))

Time: 40.23105924575 minutes



In [31]:
converted_log_data = base_dir + 'swissid_oidc_logs_april_to_sept_2019.hdf5'

# Export the data with vaex to hdf5
vaex_df.export_hdf5(path=converted_log_data, progress=True)

ERROR:MainThread:root:error creating dataset for 'response_status_code', with type dtype('O') 
Traceback (most recent call last):
  File "/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/vaex/hdf5/export.py", line 201, in export_hdf5
    array = h5column_output.require_dataset('data', shape=shape, dtype=dtype.newbyteorder(byteorder))
  File "/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/h5py/_hl/group.py", line 189, in require_dataset
    return self.create_dataset(name, *(shape, dtype), **kwds)
  File "/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/h5py/_hl/group.py", line 136, in create_dataset
    dsid = dataset.make_new_dset(self, shape, dtype, data, **kwds)
  File "/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/h5py/_hl/dataset.py", line 118, in make_new_dset
    tid = h5t.py_create(dtype, logical=1)
  File "h5py/h5t.pyx", line 1630, in h5py.h5t.py_create
  File "h5py

[########################################]:  99.76% estimated time:        0s =  0.0m =  0.0h                                      